<a href="https://colab.research.google.com/github/Jegovila/SI4/blob/main/Proyectos/Pr%C3%A1ctica_Clasificaci%C3%B3n_rgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript, Image
from IPython import display as idp
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import PIL
import io
import html
import time

# Webcam

In [ ]:
def js_to_image(js_reply):
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img
def bbox_to_bytes(bbox_array):
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))

  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# Generar Dataset

In [ ]:
sx = 200
sy = 200
ex = 400
ey = 350
colors = []

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    # loop through detections and draw them on transparent overlay image
    bbox_array = cv2.rectangle(bbox_array,(sx,sy),(ex,ey),(255,0,0),2)
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    if not js_reply:
      while True:
        filename = take_photo()
        print('Saved to {}'.format(filename))
        I = cv2.imread("photo.jpg")
        I = I[sy:ey,sx:ex]

        plt.figure(figsize=(3,4))
        plt.imshow(cv2.cvtColor(I, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

        print("z para tomar otra foto")
        print("q para salir")
        print("introduce el color")
        color_label = input()

        if color_label == "q":
          break

        elif color_label != "z":
          height, width = I.shape[:2]
          for i in range(50):
            xpos = np.random.randint(width)
            ypos = np.random.randint(height)
            b, g, r = I[ypos, xpos]
            colors.append([r, g, b, color_label])

          idp.display(plt.gcf())
          idp.clear_output(wait=True)
          break

        elif color_label == "z":
          idp.display(plt.gcf())
          idp.clear_output(wait=True)
          video_stream()
          js_reply = video_frame(label_html, bbox)
          break

      if color_label == "q":
        df = pd.DataFrame(colors, columns=['R', 'G', 'B', 'Label'])
        df.to_csv('colors.csv', index=False)
        break

# Entrenamiento

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import pickle

In [ ]:
data = pd.read_csv('colors.csv')
print(data)

En el siguiente bloque en blanco es neceario obtener los datos de entrada x/y y separarlos en x_train/test y y_train/test.

Elegir el modelo. Completar las siguientes líneas.

In [ ]:
model =                                           # <------ completar aqui
model.fit(x_train, y_train.ravel())

print('Train score',)                             # <------ completar aqui
print('Test score',)                              # <------ completar aqui

yp = model.predict(x)

print('Metricas: \n', metrics.classification_report(y,yp))
print('Confusion matrix: \n', metrics.confusion_matrix(y,yp))

pickle.dump(model,open('model.sav','wb'))

# Predicción

In [ ]:
model = pickle.load(open('model.sav', 'rb'))

In [ ]:
sx = 200
sy = 200
ex = 400
ey = 350
colors = []

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    # loop through detections and draw them on transparent overlay image
    bbox_array = cv2.rectangle(bbox_array,(sx,sy),(ex,ey),(255,0,0),2)
    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    if not js_reply:
      filename = take_photo()
      print('Saved to {}'.format(filename))
      I = cv2.imread("photo.jpg")
      I = I[sy:ey,sx:ex]

      plt.figure(figsize=(3,4))
      plt.imshow(cv2.cvtColor(I, cv2.COLOR_BGR2RGB))
      plt.axis("off")
      plt.show()

      data = []
      for i in range(3):
        channel = I[:, :, 2-i]
        indices = np.where(channel != 0)[0]
        media = np.mean(channel[indices])
        data.append(int(media))
      print(data)
      break

In [ ]:
model.predict([data])